In [14]:
from datasets import load_dataset, concatenate_datasets, Audio
from huggingface_hub import login

In [ ]:
login("")

In [3]:
dsVoiceFemale = load_dataset("ZoniaChatbot/google-colombian-spanish-female-16000-1523")
dsVoicesFemale = load_dataset("ylacombe/google-colombian-spanish", "female")

In [ ]:
print("Features:", dsVoiceFemale['train'].features)
print("Num rows:", dsVoiceFemale['train'].num_rows)
print()
print("Ejemplo:")
ejemplo_1 = dsVoiceFemale['train'][0]
for k, v in ejemplo_1.items():
    if k != 'audio':
        print(f"{k}: {v}")
    else:
        print(f"{k}: sampling_rate={v['sampling_rate']}, array_shape={len(v['array'])}")

Features: {'audio': Audio(sampling_rate=None, decode=True, num_channels=None, stream_index=None), 'text': Value('string'), 'speaker_id': Value('int64')}
Num rows: 150

Ejemplo:
   audio: sampling_rate=16000, array_shape=81920
   text: ¿Cuáles son las zonas dónde están los mejores restaurantes?
   speaker_id: 1523


In [11]:
print("Features:", dsVoicesFemale['train'].features)
print("Num rows:", dsVoicesFemale['train'].num_rows)
print()
print("Ejemplo:")
ejemplo_2 = dsVoicesFemale['train'][0]
for k, v in ejemplo_2.items():
    if k != 'audio':
        print(f"{k}: {v}")
    else:
        print(f"{k}: sampling_rate={v['sampling_rate']}, array_shape={len(v['array'])}")

Features: {'audio': Audio(sampling_rate=None, decode=True, num_channels=None, stream_index=None), 'text': Value('string'), 'speaker_id': Value('int64')}
Num rows: 2369

Ejemplo:
audio: sampling_rate=48000, array_shape=229376
text: Quiero saber qué está pasando en Veracruz.
speaker_id: 2436


In [12]:
SEED = 42
N_SAMPLES = 150

sample_A = dsVoiceFemale['train'].shuffle(seed=SEED).select(range(N_SAMPLES))
sample_B = dsVoicesFemale['train'].shuffle(seed=SEED).select(range(N_SAMPLES))

print(f"dsVoiceFemale  : {len(sample_A)}")
print(f"dsVoicesFemale : {len(sample_B)}")

dsVoiceFemale  : 150
dsVoicesFemale : 150


In [13]:
print("Features sample_A:", sample_A.features)
print("Features sample_B:", sample_B.features)

cols_A = set(sample_A.column_names)
cols_B = set(sample_B.column_names)
common_cols = list(cols_A & cols_B)
print("\nColumnas comunes:", common_cols)

sample_A = sample_A.select_columns(common_cols)
sample_B = sample_B.select_columns(common_cols)

Features sample_A: {'audio': Audio(sampling_rate=None, decode=True, num_channels=None, stream_index=None), 'text': Value('string'), 'speaker_id': Value('int64')}
Features sample_B: {'audio': Audio(sampling_rate=None, decode=True, num_channels=None, stream_index=None), 'text': Value('string'), 'speaker_id': Value('int64')}

Columnas comunes: ['speaker_id', 'text', 'audio']


In [16]:
ds_combined = concatenate_datasets([sample_A, sample_B])
ds_combined = ds_combined.shuffle(seed=SEED)

print(f"\nTotal de muestras: {len(ds_combined)}")
print(f"Features: {ds_combined.features}")


Total de muestras: 300
Features: {'speaker_id': Value('int64'), 'text': Value('string'), 'audio': Audio(sampling_rate=None, decode=True, num_channels=None, stream_index=None)}


In [18]:
for i in range(5):
    print(f"  {i}. {ds_combined[i]['text']}")

  0. Estoy buscando un restaurante de pescados y mariscos
  1. El gobierno todavía no ha opinado porque les da mucho temor dar su punto de vista sobre esta situación
  2. Tenemos diferentes recomendaciones sobre dietas de diferentes paginas y blogs en internet
  3. ¿Quiero saber el origen de la palabra ferretería?
  4. La película me llegó al corazón
